In [15]:
import os
import arcpy
arcpy.env.overwriteOutput = True

hex_level = 'hexbin_h3_r0'
directory = r"\\KVANGRAAF\e\gis_lib\vessel_traffic\global"
rasters = [f for f in os.listdir(directory) if f.endswith('.crf') and os.path.isdir(os.path.join(directory, f))]
stats = 'SUM'
print(rasters)

['ShipDensity_Commercial.crf', 'ShipDensity_Fishing.crf', 'ShipDensity_Global.crf', 'ShipDensity_Leisure.crf', 'ShipDensity_Oil_and_Gas.crf', 'ShipDensity_Passenger.crf']


In [16]:
## This function summarizes a non-multidimensional raster using SUM statistics
def hexbin_summary(hex_level, rasters, stats):
    for raster in rasters:
            print(raster)
            raster_name = raster.split('.')[0]
            print(raster_name)
            SDE_Path = r"D:\ArcGIS\Projects\Ocean and Coastal Information System\sde_on_OCIS.sde"
            table_name = os.path.join(SDE_Path, raster_name + hex_level)
            with arcpy.EnvManager(cellSize="MINOF"):
                arcpy.ia.ZonalStatisticsAsTable(
                    in_zone_data=os.path.join(SDE_Path , "ocis.sde." + hex_level + "_wgs84"),
                    zone_field="grid_id",
                    in_value_raster=os.path.join(directory, raster),
                    out_table=os.path.join(SDE_Path, raster_name + hex_level),
                    ignore_nodata="DATA",
                    statistics_type=stats,
                    process_as_multidimensional="CURRENT_SLICE",
                    percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",
                    circular_calculation="ARITHMETIC",
                    circular_wrap_value=360,
                    out_join_layer=None
                )
            arcpy.management.AlterField(
            in_table=table_name,
            field=stats,
            new_field_name=raster_name[:20].lower()+"_"+stats.lower(),
            new_field_alias=raster_name.lower()+"_"+stats.lower(),
            field_type="",
            field_length=8,
            field_is_nullable="NULLABLE",
            clear_field_alias="DO_NOT_CLEAR"
            )

In [17]:
hexbin_summary(hex_level, rasters, stats)

ShipDensity_Commercial.crf
ShipDensity_Commercial
ShipDensity_Fishing.crf
ShipDensity_Fishing
ShipDensity_Global.crf
ShipDensity_Global
ShipDensity_Leisure.crf
ShipDensity_Leisure
ShipDensity_Oil_and_Gas.crf
ShipDensity_Oil_and_Gas
ShipDensity_Passenger.crf
ShipDensity_Passenger


In [23]:
hex_level = 'hexbin_h3_r6'
hexbin_summary(hex_level, rasters, stats)

ShipDensity_Commercial.crf
ShipDensity_Commercial
ShipDensity_Fishing.crf
ShipDensity_Fishing
ShipDensity_Global.crf
ShipDensity_Global
ShipDensity_Leisure.crf
ShipDensity_Leisure
ShipDensity_Oil_and_Gas.crf
ShipDensity_Oil_and_Gas
ShipDensity_Passenger.crf
ShipDensity_Passenger


In [11]:
for raster in md_rasters:
    print(raster)
    raster_name = raster.split('.')[0]
    print(raster_name)
    SDE_Path = r"D:\ArcGIS\Projects\Ocean and Coastal Information System\sde_on_OCIS.sde"
    table_name = os.path.join(SDE_Path, raster_name + hex_level)
    with arcpy.EnvManager(cellSize="MINOF"):
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data=os.path.join(SDE_Path , "ocis.sde." + hex_level + "_wgs84"),
            zone_field="grid_id",
            in_value_raster=os.path.join(directory, raster),
            out_table=os.path.join(SDE_Path, raster_name + hex_level),
            ignore_nodata="DATA",
            statistics_type="MIN_MAX_MEAN",
            process_as_multidimensional="CURRENT_SLICE",
            percentile_values=[90],
            percentile_interpolation_type="AUTO_DETECT",
            circular_calculation="ARITHMETIC",
            circular_wrap_value=360,
            out_join_layer=None
        )
    arcpy.management.AlterField(
    in_table=table_name,
    field="min",
    new_field_name=raster_name[:20].lower()+"_min",
    new_field_alias=raster_name.lower()+"_min",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )
    arcpy.management.AlterField(
    in_table=table_name,
    field="max",
    new_field_name=raster_name[:20].lower()+"_max",
    new_field_alias=raster_name.lower()+"_max",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )
    arcpy.management.AlterField(
    in_table=table_name,
    field="mean",
    new_field_name=raster_name[:20].lower()+"_mean",
    new_field_alias=raster_name.lower()+"_mean",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )


ShipDensity_Commercial.crf
ShipDensity_Commercial
ShipDensity_Fishing.crf
ShipDensity_Fishing
ShipDensity_Global.crf
ShipDensity_Global
ShipDensity_Leisure.crf
ShipDensity_Leisure
ShipDensity_Oil_and_Gas.crf
ShipDensity_Oil_and_Gas
ShipDensity_Passenger.crf
ShipDensity_Passenger


In [3]:
hex_level = 'hexbin_h3_r1'
directory = r"\\KVANGRAAF\e\gis_lib\vessel_traffic\global"
md_rasters = [f for f in os.listdir(directory) if f.endswith('.crf') and os.path.isdir(os.path.join(directory, f))]
print(md_rasters)

['Dissolved_Inorganic_Carbon.crf', 'Dissolved_O2.crf', 'Ocean_Heat_Content_2000m.crf', 'Ocean_Heat_Content_300m.crf', 'Ocean_Heat_Content_700m.crf', 'pH.crf', 'Primary_Production.crf', 'Sea_Ice_Concentration.crf', 'Sea_Surface_Salinity.crf', 'Sea_Surface_Temperature.crf', 'Sea_Water_Bottom_Salinity.crf', 'Sea_Water_Bottom_Temperature.crf', 'Sea_Water_Salinity.crf', 'Sea_Water_Temperature.crf']


In [4]:
for raster in md_rasters:
    print(raster)
    raster_name = raster.split('.')[0]
    print(raster_name)
    SDE_Path = r"D:\ArcGIS\Projects\Ocean and Coastal Information System\sde_on_OCIS.sde"
    table_name = os.path.join(SDE_Path, raster_name + hex_level)
    with arcpy.EnvManager(cellSize="MINOF"):
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data=os.path.join(SDE_Path , "ocis.sde." + hex_level + "_wgs84"),
            zone_field="grid_id",
            in_value_raster=os.path.join(directory, raster),
            out_table=os.path.join(SDE_Path, raster_name + "_" + hex_level),
            ignore_nodata="DATA",
            statistics_type="MIN_MAX_MEAN",
            process_as_multidimensional="ALL_SLICES",
            percentile_values=[90],
            percentile_interpolation_type="AUTO_DETECT",
            circular_calculation="ARITHMETIC",
            circular_wrap_value=360,
            out_join_layer=None
        )
    arcpy.management.AlterField(
    in_table=table_name,
    field="min",
    new_field_name=raster_name[:20].lower()+"_min",
    new_field_alias=raster_name.lower()+"_min",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )
    arcpy.management.AlterField(
    in_table=table_name,
    field="max",
    new_field_name=raster_name[:20].lower()+"_max",
    new_field_alias=raster_name.lower()+"_max",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )
    arcpy.management.AlterField(
    in_table=table_name,
    field="mean",
    new_field_name=raster_name[:20].lower()+"_mean",
    new_field_alias=raster_name.lower()+"_mean",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )

Dissolved_Inorganic_Carbon.crf
Dissolved_Inorganic_Carbon
Dissolved_O2.crf
Dissolved_O2
Ocean_Heat_Content_2000m.crf
Ocean_Heat_Content_2000m
Ocean_Heat_Content_300m.crf
Ocean_Heat_Content_300m
Ocean_Heat_Content_700m.crf
Ocean_Heat_Content_700m
pH.crf
pH
Primary_Production.crf
Primary_Production
Sea_Ice_Concentration.crf
Sea_Ice_Concentration
Sea_Surface_Salinity.crf
Sea_Surface_Salinity
Sea_Surface_Temperature.crf
Sea_Surface_Temperature
Sea_Water_Bottom_Salinity.crf
Sea_Water_Bottom_Salinity
Sea_Water_Bottom_Temperature.crf
Sea_Water_Bottom_Temperature
Sea_Water_Salinity.crf
Sea_Water_Salinity
Sea_Water_Temperature.crf
Sea_Water_Temperature


In [ ]:
hex_level = 'hexbin_h3_r2'
directory = r"\\KVANGRAAF\e\gis_lib\vessel_traffic\global"
md_rasters = [f for f in os.listdir(directory) if f.endswith('.crf') and os.path.isdir(os.path.join(directory, f))]
print(md_rasters)

In [ ]:
for raster in md_rasters:
    print(raster)
    raster_name = raster.split('.')[0]
    print(raster_name)
    SDE_Path = r"D:\ArcGIS\Projects\Ocean and Coastal Information System\sde_on_OCIS.sde"
    table_name = os.path.join(SDE_Path, raster_name + hex_level)
    with arcpy.EnvManager(cellSize="MINOF"):
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data=os.path.join(SDE_Path , "ocis.sde." + hex_level + "_wgs84"),
            zone_field="grid_id",
            in_value_raster=os.path.join(directory, raster),
            out_table=os.path.join(SDE_Path, raster_name + "_" + hex_level),
            ignore_nodata="DATA",
            statistics_type="MIN_MAX_MEAN",
            process_as_multidimensional="ALL_SLICES",
            percentile_values=[90],
            percentile_interpolation_type="AUTO_DETECT",
            circular_calculation="ARITHMETIC",
            circular_wrap_value=360,
            out_join_layer=None
        )
    arcpy.management.AlterField(
    in_table=table_name,
    field="min",
    new_field_name=raster_name[:20].lower()+"_min",
    new_field_alias=raster_name.lower()+"_min",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )
    arcpy.management.AlterField(
    in_table=table_name,
    field="max",
    new_field_name=raster_name[:20].lower()+"_max",
    new_field_alias=raster_name.lower()+"_max",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )
    arcpy.management.AlterField(
    in_table=table_name,
    field="mean",
    new_field_name=raster_name[:20].lower()+"_mean",
    new_field_alias=raster_name.lower()+"_mean",
    field_type="",
    field_length=8,
    field_is_nullable="NULLABLE",
    clear_field_alias="DO_NOT_CLEAR"
    )